In [1]:
!pip install -q -U tensorflow-addons

     |████████████████████████████████| 1.1MB 3.3MB/s 


In [2]:
__author__ = 'Alexander Soroka, soroka.a.m@gmail.com'
__copyright__ = """Copyright 2020 Alexander Soroka"""


import argparse
import glob
import numpy as np
import tensorflow as tf
import time
from tensorflow.python import keras as keras
from tensorflow.python.keras.callbacks import LearningRateScheduler

import tensorflow_addons as tfa

LOG_DIR = 'logs'
SHUFFLE_BUFFER = 4
BATCH_SIZE = 32
NUM_CLASSES = 6
PARALLEL_CALLS=4
RESIZE_TO = 224
TRAINSET_SIZE = 14034
VALSET_SIZE = 3000


def parse_proto_example(proto):
    keys_to_features = {
        'image/encoded': tf.io.FixedLenFeature((), tf.string, default_value=''),
        'image/class/label': tf.io.FixedLenFeature([], tf.int64, default_value=tf.zeros([], dtype=tf.int64))
    }
    example = tf.io.parse_single_example(proto, keys_to_features)
    example['image'] = tf.image.decode_jpeg(example['image/encoded'], channels=3)
    example['image'] = tf.image.convert_image_dtype(example['image'], dtype=tf.float32)
    example['image'] = tf.image.resize(example['image'], tf.constant([RESIZE_TO, RESIZE_TO]))
    return example['image'], tf.one_hot(example['image/class/label'], depth=NUM_CLASSES)


def normalize(image, label):
    return tf.image.per_image_standardization(image), label

def resize(image, label):
    return tf.image.resize(image, tf.constant([RESIZE_TO, RESIZE_TO])), label

def augmentation(image, label):
    image = tf.image.convert_image_dtype(image, tf.float32)
    image = tf.image.random_flip_left_right(image)
    degree = 30
    degrand = np.random.uniform(-degree, degree)
    image = tfa.image.rotate(image, np.pi * degrand / 180, interpolation='BILINEAR')
    image = tf.image.random_brightness(image, 0.1, seed=None)
    image = tf.image.random_contrast(image, 0.7, 1.4, seed=None)
    noise_img = tf.random.normal(shape=tf.shape(image), mean=0.0, stddev=1.0, dtype=tf.float32)
    noise_img = tf.add(image, noise_img)
    return image,label

def create_dataset(filenames, batch_size):
    """Create dataset from tfrecords file
    :tfrecords_files: Mask to collect tfrecords file of dataset
    :returns: tf.data.Dataset
    """
    return tf.data.TFRecordDataset(filenames)\
        .map(parse_proto_example)\
        .map(resize)\
        .map(normalize)\
        .batch(batch_size)\
        .prefetch(batch_size)

def create_aug_dataset(filenames, batch_size):
    """Create dataset from tfrecords file
    :tfrecords_files: Mask to collect tfrecords file of dataset
    :returns: tf.data.Dataset
    """
    return tf.data.TFRecordDataset(filenames)\
        .map(parse_proto_example)\
        .map(resize)\
        .map(normalize)\
        .map(augmentation)\
        .map(normalize)\
        .shuffle(buffer_size=5 * batch_size)\
        .batch(batch_size)\
        .prefetch(2 * batch_size)


def build_model():
    new_model = tf.keras.applications.MobileNetV2(input_shape=(224,224,3),
						                                      classes = NUM_CLASSES,
                                                  include_top=False,
                                                  weights='imagenet')
    

    return tf.keras.models.Sequential([
    	new_model,
      tf.keras.layers.GlobalAveragePooling2D(),
      tf.keras.layers.Dropout(0.5),
    	tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(NUM_CLASSES, activation=tf.keras.activations.softmax)
    ])


def main():
    #args = argparse.ArgumentParser()
    #args.add_argument('--train', type=str, help='Glob pattern to collect train tfrecord files')
    #args.add_argument('--test', type=str, help='Glob pattern to collect test tfrecord files')
    #args = args.parse_args()
    train_path = '/content/drive/My Drive/SMOMI/dataset/train*'
    test_path = '/content/drive/My Drive/SMOMI/dataset/val*'

    train_dataset = create_aug_dataset(glob.glob(train_path), BATCH_SIZE)
    validation_dataset = create_dataset(glob.glob(test_path), BATCH_SIZE)

    model = build_model()

    model.compile(
        optimizer=tf.optimizers.Adam(lr=8e-6),
        loss=tf.keras.losses.categorical_crossentropy,
        metrics=[tf.keras.metrics.categorical_accuracy],
    )

    weights_file = "/content/drive/My Drive/SMOMI/w-lab4_5.hdf5"
    log_dir='/content/drive/My Drive/SMOMI/{}/lab4_5-new/ilcd-{}'.format(LOG_DIR, time.time())
    model.fit(
        train_dataset,
        epochs=100,
        validation_data=validation_dataset,
        callbacks=[
            tf.keras.callbacks.TensorBoard(log_dir),
            tf.keras.callbacks.ModelCheckpoint(filepath=weights_file, monitor='val_categorical_accuracy', mode='max', save_best_only=True, save_weights_only=True, verbose=1)
        ]
    )


if __name__ == '__main__':
    main()

9412608/9406464 [==============================] - 0s 0us/step
Epoch 1/100
      1/Unknown - 0s 624us/step - loss: 3.5457 - categorical_accuracy: 0.0000e+00WARNING:tensorflow:From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
      2/Unknown - 0s 136ms/step - loss: 3.3799 - categorical_accuracy: 0.0312    WARNING:tensorflow:Callbacks method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0670s vs `on_train_batch_end` time: 0.2028s). Check your callbacks.
    439/Unknown - 100s 228ms/step - loss: 2.1170 - categorical_accuracy: 0.1522
Epoch 00001: val_categorical_accuracy improved from -inf to 0.16500, saving model to /content/drive/My Drive/SMOMI/w-lab4_5.hdf5
439/439 [==============================] - 118s 269ms/step - loss: 2.1170 - categorical_accuracy: 0.1

KeyboardInterrupt: ignored